In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer


df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [2]:
# Agrupaciones

# Zona Provincias
provincias_sur = ['Yucatán','Quintana Roo','Campeche','Tabasco', 'Veracruz', 'Oaxaca', 'Chiapas', 'Guerrero', 'Michoacán']
provincias_centro = ['Tlaxcala','Morelos','Puebla','Hidalgo','Aguascalientes', 'Distrito Federal', 'Jalisco', 'Edo. de México','Querétaro', 'Colima', 'Nayarit', 'Aguascalientes', 'Guanajuato', 'Hidalgo']
provincias_norte = ['Sonora','San luis Potosí','Baja California Norte', 'Baja California Sur', 'Chihuahua', 'Durango', 'Coahuila', 'Sinaloa', 'Nuevo León',  'Tamaulipas']

prov_por_precio = {'Distrito Federal':0, 'Edo. de México':1, 'Nuevo León':2, 'Guerrero':3, 'Quintana Roo':4, 'Jalisco':5, 'Tabasco':6, 'Puebla':6, 'Morelos':6, 'Querétaro':7, 'Yucatán':8, 'Chiapas':9, 'San luis Potosí':9, 'Veracruz':10, 'Guanajuato':11, 'Chihuahua':12, 'Sinaloa':13, 'Tamaulipas':14, 'Oaxaca':14, 'Campeche':14, 'Colima':15, 'Coahuila':15, 'Sonora':16, 'Michoacán':16, 'Aguascalientes':16, 'Zacatecas':17, 'Nayarit':18, 'Hidalgo':19, 'Baja California Sur':20, 'Durango':20,'Tlaxcala':21, 'Baja California Norte':22 ,'otro':23}
def top_provincias(x):
    if x in prov_por_precio:
        return prov_por_precio[x]
    else:
        return 24

# Intervalos metros
punto20 = df_train['metrostotales'].quantile(0.2)
punto40 = df_train['metrostotales'].quantile(0.4)
punto60 = df_train['metrostotales'].quantile(0.6)
punto80 = df_train['metrostotales'].quantile(0.8)
punto20_cubiertos = df_train['metroscubiertos'].quantile(0.2)
punto40_cubiertos = df_train['metroscubiertos'].quantile(0.4)
punto60_cubiertos = df_train['metroscubiertos'].quantile(0.6)
punto80_cubiertos = df_train['metroscubiertos'].quantile(0.8)

# Division de tipo de propiedad
propiedades_compartidas = ['Casa en condominio', 'Duplex', 'Departamento Compartido','Lote']
propiedades_frecuentes = ['Casa', 'Apartamento', 'Edificio']

# Escuelas y Centros Comerciales Cercanos
def cercanos(x):
    if not x['escuelascercanas'] and not x['centroscomercialescercanos']:
        return 'Ninguno'
    if x['escuelascercanas'] and not x['centroscomercialescercanos']:
        return 'Centro Comerciales'
    if not x['centroscomercialescercanos'] and x['escuelascercanas']:
        return 'Escuelas Cercanas'
    return 'Ambos'

# Delincuencia
ciudades_mas_peligrosas = pd.read_csv('./data/ciudades_mas_peligrosas.csv')
df_crimenes = pd.merge(df_train, ciudades_mas_peligrosas, on = 'ciudad', how= 'left')

# Turismo
ciudades_mas_turisticas = pd.read_csv('./data/ciudades_mas_turisticas.csv')
df_turismo = pd.merge(df_train, ciudades_mas_turisticas, on = 'ciudad', how= 'left')

# Personas por propiedad
def calculo_inquilinos(x):
    if (x['habitaciones'] == np.nan) or (x['banos'] == np.nan) or (x['habitaciones'] == 0) or (x['banos'] == 0):
        return 0
    if (x['habitaciones'] == 1) and (x['banos'] == 1):
        return 1
    if (x['habitaciones'] == 2) and (x['banos'] == 1):
        return 3
    if (x['habitaciones'] == 2) and (x['banos'] == 2):
        return 4
    if (x['habitaciones'] == 3) and (x['banos'] == 1):
        return 5
    if (x['habitaciones'] == 3) and (x['banos'] == 2):
        return 6
    if (x['habitaciones'] == 4) and (x['banos'] == 1):
        return 6
    if (x['habitaciones'] == 4) and (x['banos'] == 2):
        return 7
    if (x['habitaciones'] == 4) and ((x['banos'] == 3) or (x['banos'] == 4)):
        return 8
    if (x['habitaciones'] == 5):
        return 10
    if (x['habitaciones'] > 5):
        return 11
    else:
        return 9

# Ciudad Centrica por provincia
ciudades_centricas = pd.read_csv('./data/ciudadcentrica_por_provincia.csv')
df_centro = pd.merge(df_train, ciudades_centricas, on = 'ciudad', how= 'left')

In [3]:
# Insight
zona = df_train
totales2 = zona['metrostotales'].quantile(0.2)
totales4 = zona['metrostotales'].quantile(0.4)
totales6 = zona['metrostotales'].quantile(0.6)
totales8 = zona['metrostotales'].quantile(0.8)
cubiertos2 = zona['metroscubiertos'].quantile(0.2)
cubiertos4 = zona['metroscubiertos'].quantile(0.4)
cubiertos6 = zona['metroscubiertos'].quantile(0.6)
cubiertos8 = zona['metroscubiertos'].quantile(0.8)

In [4]:
zona_metros = zona.groupby('metrostotales').agg('mean').reset_index()
zona_metros_cub = zona.groupby('metroscubiertos').agg('mean').reset_index()

zona_metros['grupoMetros'] = list(map(lambda x: 'Grupo 1' if x <= totales2 else ('Grupo 2' if totales2<x<=totales4 else ('Grupo 3' if totales4<x<=totales6 else ('Grupo 4' if totales6<x<=totales8 else 'Grupo 5'))), zona_metros["metrostotales"]))
zona_metros_un = zona_metros.unstack()

zona_metros_cub['grupoCubiertos'] = list(map(lambda x: 'Grupo 1' if x <= cubiertos2 else ('Grupo 2' if cubiertos2<x<=cubiertos4 else ('Grupo 3' if cubiertos4<x<=cubiertos6 else ('Grupo 4' if cubiertos6<x<=cubiertos8 else 'Grupo 5'))), zona_metros_cub["metroscubiertos"]))
zona_metros_cub_un = zona_metros_cub.unstack()

In [5]:
def map_values(vals, m):
    res = []
    for x in vals:
        res.append(m(x))
    return res
    

def features_independientes_precio(df):
    '''Devuelve una copia del dataframe con nuevas columnas, 
    donde los datos generados no dependen del precio'''
    df = df.copy()
    
    # Por metros totales  cubiertos
    df['metrostotales'] = df['metrostotales'].fillna(df['metroscubiertos'])
    df['metroscubiertos'] = df['metroscubiertos'].fillna(df['metrostotales'])
    df['porcentaje_metros'] = df['metroscubiertos']/df['metrostotales']
    df['diferencia_metros'] = df['metrostotales'] - df['metroscubiertos']
    df['intervalo_metros_totales'] = map_values(df['metrostotales'].values, lambda x: 'Grupo1' if x<punto20 else ('Grupo2' if punto20<x<punto40 else ('Grupo3' if punto40<x<punto60 else('Grupo4' if punto60<x<punto80 else 'Grupo5'))))
    df['intervalo_metros_cubiertos'] = map_values(df['metroscubiertos'].values, lambda x: 'Grupo1' if x<punto20_cubiertos else ('Grupo2' if punto20_cubiertos<x<punto40_cubiertos else ('Grupo3' if punto40_cubiertos<x<punto60_cubiertos else('Grupo4' if punto60_cubiertos<x<punto80_cubiertos else 'Grupo5'))))

    
    # Por Tipo de propiedad
    df['tipodepropiedad'] = df['tipodepropiedad'].fillna('otro') # Otra forma de llenar los nans para el tipo de prop?
    df['escomercial'] = df.tipodepropiedad.str.contains('.*omercial.*').astype(bool)
    df['promedio_metros_tipo_propiedad'] = df.groupby(['tipodepropiedad'])['metrostotales'].transform('mean')
    df['promedio_metros_cub_tipo_propiedad'] = df.groupby(['tipodepropiedad'])['metroscubiertos'].transform('mean')
    df['tipo_propiedad_compartida'] = map_values(df['tipodepropiedad'].values, lambda x: True if x in propiedades_compartidas else False)
    df['prop_frecuente'] = map_values(df['tipodepropiedad'].values, lambda x: True if x in propiedades_frecuentes else False)     

    # Por ubicación
    df['provincia'] = df['provincia'].fillna('otro') # Otra forma de llenar los nans para el tipo de prop?
    df['zona'] = map_values(df['provincia'].values, lambda x: 'Norte ' if x in provincias_norte else ('Sur' if x in provincias_sur else 'Centro'))
    df['top_provincia'] = df['provincia'].apply(top_provincias)
    df['es_ciudad_centrica'] = df_centro['centro'].replace({np.nan:False})
    df['promedio_metros_totales_provincia'] = df.groupby(['provincia'])['metrostotales'].transform('mean')
    df['promedio_metros_cubiertos_provincia'] = df.groupby(['provincia'])['metroscubiertos'].transform('mean')
    
  
    # Por fecha
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['anio'] = df['fecha'].dt.year
    df['mes'] = df['fecha'].dt.month
    df['dia'] = df['fecha'].dt.day
    df['trimestre'] = (df['fecha'].dt.month)%3+1
    df['fecha'] = df['fecha'].astype(int)
    
    # Propiedades booleanas
    df['escualas_centros_cercanos'] = df[['centroscomercialescercanos', 'escuelascercanas']].apply(cercanos, axis=1)
    df['delincuencia'] = df_crimenes['peligro'].replace({np.nan:False})
    df['turismo'] = df_turismo['turismo'].replace({np.nan:False})
    df['es_antigua'] = map_values(df['antiguedad'].values, lambda x: True if x>30 else False)

    # Habitaciones, garages y banos
    df['cantidad_inquilinos'] = df[['habitaciones', 'banos']].apply(calculo_inquilinos,axis=1)

    df['metros_totales_normalizados'] = df['metrostotales']/(df['metrostotales']).mean()
    df['metros_cubiertos_normalizados'] = df['metroscubiertos']/(df['metroscubiertos']).mean()
    
    return df

In [6]:
def generalizar_feature(df, f_nombre, f_reemplazante, f_count, N):
    '''Dado un dataframe `df`, el nombre de los features `f_nombre`, `f_reemplazante` y `f_count`,
    y una cantidad N, sobreescribe la columna de `f_nombre` por los valores en `f_reemplazante`
    a todas las filas cuya cuenta de `f_count` sea menor a N.'''

    df.loc[df.groupby(f_count)[f_count].transform('size') < N, f_nombre] = df[f_reemplazante]


def trasladar_features(df, df_train, features, f_label, f_target, function):
    '''Dados dos datafarmes `df`, `df_train`, una lista de `features`, dos features
    `f_label` y `f_target`, y una funcion de groupby `function`, agrupa el `df_train` por
    los `features`, transformando por `function` a f_target y traslada estos valores a `df`'''

    len_df = len(df.values)
    df_train[f_target] = df_train.groupby(features)[f_label].transform(function)
    df_promedio = df_train.groupby(features).agg({f_label:function}).rename(columns = {f_label: f_target}).reset_index()
    return pd.merge(df, df_promedio, on=features, how='left')
    
    
def features_dependientes_precio(df, df_train):
    '''Devuelve una copia del dataframe df con nuevas columnas dependientes del precio,
    usando el dataframe df_train con los precios para generar los datos.
    El dataframe df puede ser el mismo que df_train.'''
    df = df.copy()
    df_train = df_train.copy()
    
    # Por provincia
    df = trasladar_features(df, df_train, 'provincia', 'precio', 'promedio_precio_provincia', 'mean')
    df['promedio_precio_provincia'] = df['promedio_precio_provincia'].fillna(df_train['promedio_precio_provincia'].mean())
    
    # Por ciudad
    df = trasladar_features(df, df_train, ['provincia', 'ciudad'], 'precio', 'promedio_precio_ciudad', 'mean')
    df['promedio_precio_ciudad'] = df['promedio_precio_ciudad'].fillna(df_train['promedio_precio_ciudad'].mean())
    
    df['promedio_precio_ciudad_gen'] = df['promedio_precio_ciudad']
    generalizar_feature(df, 'promedio_precio_ciudad_gen', 'promedio_precio_provincia', 'provincia', 10)
    
    df = trasladar_features(df, df_train, ['provincia', 'ciudad'], 'precio', 'varianza_precio_ciudad', 'std')
    df['varianza_precio_ciudad'] = df['varianza_precio_ciudad'].fillna(df_train['varianza_precio_ciudad'].mean())
    
    df = trasladar_features(df, df_train, ['provincia', 'ciudad'], 'precio', 'count_ciudad', 'count')
    df['count_ciudad'] = df['count_ciudad'].fillna(df['count_ciudad'].mean())
       
    
    # Por id_zona
    df = trasladar_features(df, df_train, 'idzona', 'precio', 'promedio_id_zona', 'mean')
    df['promedio_id_zona'] = df['promedio_id_zona'].fillna(df['promedio_precio_ciudad'])
    
    df['promedio_id_zona_gen'] = df['promedio_id_zona']
    generalizar_feature(df, 'promedio_id_zona_gen', 'promedio_precio_ciudad_gen', 'idzona', 10)

    df = trasladar_features(df, df_train, 'idzona', 'precio', 'varianza_id_zona', 'std')
    df['varianza_id_zona'] = df['varianza_id_zona'].fillna(df['varianza_precio_ciudad'])
    
    df = trasladar_features(df, df_train, 'idzona', 'precio', 'count_id_zona', 'count')
    df['count_id_zona'] = df['count_id_zona'].fillna(df['count_ciudad'])


    # Por tipo de propiedad
    df_train['tipodepropiedad'] = df_train['tipodepropiedad'].fillna('otro') # Otra forma de llenar los nans para el tipo de prop?
    df['tipodepropiedad'] = df['tipodepropiedad'].fillna('otro')
    df = trasladar_features(df, df_train, 'tipodepropiedad', 'precio', 'promedio_precio_tipo_propiedad', 'mean')
    df['promedio_precio_tipo_propiedad'] = df['promedio_precio_tipo_propiedad'].fillna(df['promedio_precio_tipo_propiedad'].mean())
    
    df = trasladar_features(df, df_train, ['ciudad', 'tipodepropiedad'], 'precio', 'promedio_precio_tipo_propiedad_ciudad', 'mean')
    df['promedio_precio_tipo_propiedad_ciudad'] = df['promedio_precio_tipo_propiedad_ciudad'].fillna(df['promedio_precio_tipo_propiedad'])
    df['promedio_precio_tipo_propiedad_ciudad_gen'] = df['promedio_precio_tipo_propiedad_ciudad']
    generalizar_feature(df, 'promedio_precio_tipo_propiedad_ciudad_gen', 'promedio_precio_tipo_propiedad', 'ciudad', 100)
    
    df = trasladar_features(df, df_train, 'tipodepropiedad', 'precio', 'count_tipo_propiedad', 'count')
    df['count_tipo_propiedad'] = df['count_tipo_propiedad'].fillna(0)
    
    df = trasladar_features(df, df_train, ['ciudad', 'tipodepropiedad'], 'precio', 'count_tipo_propiedad_ciudad', 'count')
    df['count_tipo_propiedad_ciudad'] = df['count_tipo_propiedad_ciudad'].fillna(df['count_ciudad'])
       
    # Por fecha
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['anio'] = df['fecha'].dt.year
    df['mes'] = df['fecha'].dt.month
    df['dia'] = df['fecha'].dt.day
    df['fecha'] = df['fecha'].astype(int)

    df_train['fecha'] = pd.to_datetime(df_train['fecha'])
    df_train['anio'] = df_train['fecha'].dt.year
    df_train['mes'] = df_train['fecha'].dt.month
    
    df = trasladar_features(df, df_train, ['anio', 'mes'], 'precio', 'promedio_por_mes', 'mean')
    df['promedio_por_mes'] = df['promedio_por_mes'].fillna(df_train['promedio_por_mes'].mean())
    
    df = trasladar_features(df, df_train, ['anio', 'mes'], 'precio', 'varianza_por_mes', 'std')
    df['varianza_por_mes'] = df['varianza_por_mes']#.fillna(df_train['varianza_por_mes'].std())
    
    # Por cantidad de habitaciones, banos y garages


    df = trasladar_features(df, df_train, ['habitaciones'], 'precio', 'promedio_precio_habitaciones', 'mean')
    df = trasladar_features(df, df_train, ['habitaciones', 'banos', 'garages'], 'precio', 'promedio_precio_habitaciones_banos_garages', 'mean')
    df = trasladar_features(df, df_train, ['banos', 'garages'], 'precio', 'promedio_precio_banos_garages', 'mean')
    df = trasladar_features(df, df_train, ['tipodepropiedad', 'banos', 'habitaciones', 'garages'], 
        'precio', 'promedio_precio_hbg_tipo_propiedad', 'mean')
    df = trasladar_features(df, df_train, ['provincia', 'tipodepropiedad', 'banos', 'habitaciones', 'garages'], 
        'precio', 'promedio_precio_hbg_tipo_propiedad_provincia', 'mean')
    df['promedio_precio_habitaciones'] = df['promedio_precio_habitaciones'].fillna(df_train['promedio_precio_habitaciones'].mean())
    df['promedio_precio_habitaciones_banos_garages'] = df['promedio_precio_habitaciones_banos_garages'].fillna(df_train['promedio_precio_habitaciones'].mean())
    df['promedio_precio_banos_garages'] = df['promedio_precio_banos_garages'].fillna(df_train['promedio_precio_banos_garages'].mean())
    df['promedio_precio_hbg_tipo_propiedad'] = df['promedio_precio_hbg_tipo_propiedad'].fillna(df_train['promedio_precio_hbg_tipo_propiedad'].mean())
    df['promedio_precio_hbg_tipo_propiedad_provincia'] = df['promedio_precio_hbg_tipo_propiedad_provincia'].fillna(df['promedio_precio_hbg_tipo_propiedad'])
    generalizar_feature(df, 'promedio_precio_hbg_tipo_propiedad_provincia_gen', 'promedio_precio_hbg_tipo_propiedad_provincia', 'ciudad', 30)
    
    

        
    # Por propiedades booleanas

    df = trasladar_features(df, df_train, ['gimnasio', 'usosmultiples', 'piscina'], 'precio', 'promedio_precio_booleanos', 'mean')
    df['promedio_precio_booleanos'] = df['promedio_precio_booleanos'].fillna(df_train['promedio_precio_booleanos'].mean())
        
    # Insight    
    df['puntaje'] = df_puntajes(df)
    
    return df


In [7]:
def features_de_csvs(df_train, df_test):
    df_train_similares = pd.read_csv('data/train_similares.csv')
    df_train = pd.merge(df_train, df_train_similares, on='id')
    df_test_similares = pd.read_csv('data/test_similares.csv')
    df_test = pd.merge(df_test, df_test_similares, on='id')
    
    return df_train, df_test

In [8]:
def features_booleanas (df):
    df['es_casa'] = (df['tipodepropiedad'] == 'Casa')
    df['es_apartamento'] = (df['tipodepropiedad'] == 'Apartamento')
    df['es_capital'] = (df['provincia'] == 'Distrito Federal')
    df['es_turismo'] = (df['provincia'] == 'Quintana Roo')
    df['es_lujoso'] = (df['pisina'] == 1.0 & df['usosmultiplos'] == 1.0 & df['gimnasio'] == 1.0)

In [9]:
# Garages
zonaPromedioGarages = df_train.groupby(['tipodepropiedad', 'habitaciones']).agg({'garages':'mean'}).reset_index()
df_train['garages'] = df_train['garages'].fillna(zonaPromedioGarages['garages'])

promedioGarages = zonaPromedioGarages['garages'].mean()
df_train['garages'] = df_train['garages'].fillna(promedioGarages)

# Habitaciones
zonaPromedioHabitaciones = df_train.groupby(['tipodepropiedad', 'provincia']).agg({'habitaciones':'mean'}).reset_index()
df_train['habitaciones'] = df_train['habitaciones'].fillna(zonaPromedioHabitaciones['habitaciones'])

promedioHabitaciones = zonaPromedioHabitaciones['habitaciones'].mean()
df_train['habitaciones'] = df_train['habitaciones'].fillna(promedioHabitaciones)

# Banos
zonaPromedioBanos = df_train.groupby(['tipodepropiedad', 'provincia']).agg({'banos':'mean'}).reset_index()
df_train['banos'] = df_train['banos'].fillna(zonaPromedioBanos['banos'])

promedioBanos = zonaPromedioBanos['banos'].mean()
df_train['banos'] = df_train['banos'].fillna(promedioBanos)

In [10]:
provincias1 = ['Guerrero', 'Tabasco', 'Nueva León', 'Quintana Roo', 'Morelos', 'Jalisco', 'Puebla']
provincias2 = ['Querétaro', 'Yucatán', 'Chiapas', 'Guanajuato', 'San Luis Potosí', 'Veracruz', 'Chihuahua', 'Sinaloa']
provincias3 = ['Tamaulipas', 'Oaxaca', 'Campeche', 'Campeche', 'Zacatecas', 'Colima', 'Aguascalientes', 'Michoacán', 'Sonora', 'Nayarit'] 

def df_puntajes(zona):
    attrs = ['piscina', 'usosmultiples', 'gimnasio', 'banos', 'habitaciones', 
                      'ciudad', 'metrostotales', 'metroscubiertos', 'provincia']
    zona_vals = zona[attrs].values
    puntaje_vals = []
    for prop in zona_vals:
        puntaje = 0
        if prop[attrs.index('piscina')] == 1.0:
            puntaje += 8
        if prop[attrs.index('usosmultiples')] == 1.0:
            puntaje +=5
        if prop[attrs.index('gimnasio')] == 1.0:
            puntaje +=5
        if prop[attrs.index('banos')] == 2.0:
            puntaje +=2
        elif prop[attrs.index('banos')] == 3.0:
            puntaje += 5
        elif prop[attrs.index('banos')] == 4.0:
            puntaje += 6
        if prop[attrs.index('habitaciones')] == 2.0:
            puntaje += 2
        elif prop[attrs.index('habitaciones')] == 3.0:
            puntaje += 5
        elif prop[attrs.index('habitaciones')] > 3.0:
            puntaje += 6
            
        x = prop[attrs.index('metrostotales')]
        grupo_metros_totales = 'Grupo 1' if x <= totales2 else ('Grupo 2' if totales2<x<=totales4 else ('Grupo 3' if totales4<x<=totales6 else ('Grupo 4' if totales6<x<=totales8 else 'Grupo 5')))
        if grupo_metros_totales == 'Grupo 1':
            puntaje += 0
        elif grupo_metros_totales == 'Grupo 2':
            puntaje += 4
        elif grupo_metros_totales == 'Grupo 3':
            puntaje += 6
        elif grupo_metros_totales == 'Grupo 4':
            puntaje += 8
        elif grupo_metros_totales == 'Grupo 5':
            puntaje += 10
        
        x = prop[attrs.index('metroscubiertos')]
        grupo_metros_cubiertos = 'Grupo 1' if x <= cubiertos2 else ('Grupo 2' if cubiertos2<x<=cubiertos4 else ('Grupo 3' if cubiertos4<x<=cubiertos6 else ('Grupo 4' if cubiertos6<x<=cubiertos8 else 'Grupo 5')))
        if grupo_metros_cubiertos == 'Grupo 1':
            puntaje +=0
        elif grupo_metros_cubiertos == 'Grupo 2':
            puntaje += 4
        elif grupo_metros_cubiertos == 'Grupo 3':
            puntaje += 6
        elif grupo_metros_cubiertos == 'Grupo 4':
            puntaje += 8
        elif grupo_metros_cubiertos == 'Grupo 5':
            puntaje += 10
        if prop[attrs.index('provincia')] == 'Distrito Federal':
            puntaje += 8
        elif prop[attrs.index('provincia')] == 'Edo. de México':
            puntaje += 7
        elif prop[attrs.index('provincia')] in provincias1:
            puntaje += 5
        elif prop[attrs.index('provincia')] in provincias2:
            puntaje += 4
        elif prop[attrs.index('provincia')] in provincias3:
            puntaje += 2
        puntaje_vals.append(puntaje)
    return puntaje_vals


In [11]:
# from sklearn.neighbors import KDTree
# import pandas as pd
# import numpy as np

# df_train = pd.read_csv('data/train.csv')

In [12]:
# df_ohe = pd.get_dummies(df_train[['provincia', 'tipodepropiedad', 'habitaciones', 'garages', 
#                    'banos', 'gimnasio', 'usosmultiples', 'piscina', 
#                    'escuelascercanas', 'centroscomercialescercanos']])

# df_ohe.head(10)

In [13]:
# tree = KDTree(df_ohe, leaf_size=200)

In [14]:
# df_test = pd.read_csv('data/test.csv')
# df_test_f = df_test[['provincia', 'tipodepropiedad', 'habitaciones', 'garages', 
#                    'banos', 'gimnasio', 'usosmultiples', 'piscina', 
#                    'escuelascercanas', 'centroscomercialescercanos']]
# df_train_f = df_train[['provincia', 'tipodepropiedad', 'habitaciones', 'garages', 
#                    'banos', 'gimnasio', 'usosmultiples', 'piscina', 
#                    'escuelascercanas', 'centroscomercialescercanos']]

# df_full = pd.concat([df_test_f, df_train_f], ignore_index=True)


# df_ohe_test = pd.get_dummies(df_full).iloc[0:60000]
# df_ohe_test = df_ohe_test.fillna(0)

In [15]:
# KDTree.valid_metrics

In [16]:
# display(df_ohe_test.shape)
# display(df_ohe.shape)
# dists = tree.query_radius(df_ohe_test.values[:60000], r=0, count_only=True)
# dists

In [17]:
# df_test['similares_count'] = dists
# df_test[['id', 'similares_count']].to_csv('data/test_similares.csv', index=False)

In [18]:
# df_ohe = df_ohe.fillna(0)
# display(df_ohe_test.shape)
# display(df_ohe.shape)
# dists = tree.query_radius(df_ohe.values, r=0, count_only=True)
# dists

In [19]:
# df_train['similares_count'] = dists
# df_train[['id', 'similares_count']].to_csv('data/train_similares.csv', index=False)

In [20]:
# df_test.iloc[1]

In [21]:
# df_train[(df_train['gimnasio'] == 0) & (df_train['usosmultiples'] == 0) & (df_train['piscina'] == 0)
#         & (df_train['escuelascercanas'] == 0) & (df_train['centroscomercialescercanos'] == 0)
#         & (df_train['provincia'] == 'Yucatán') & (df_train['tipodepropiedad'] == 'Apartamento')
#         & (df_train['habitaciones'] == 1) & (df_train['banos'] == 1) & (df_train['garages'] == 1)]

In [22]:
# ind = features_independientes_precio(df_train)

In [23]:
# ind[['escomercial',
#        'promedio_metros_tipo_propiedad', 'tipo_propiedad_compartida',
#        'prop_frecuente', 'zona','top_provincia', 'es_ciudad_centrica',
#        'promedio_metros_totales_provincia',
#        'promedio_metros_cubiertos_provincia', 'porcentaje_metros',
#        'diferencia_metros', 'intervalo_metros_totales',
#        'intervalo_metros_cubiertos', 'anio', 'mes', 'dia', 'trimestre',
#        'escualas_centros_cercanos', 'delincuencia', 'turismo', 'es_antigua', 'cantidad_inquilinos']].head()

In [24]:
# ind_test = features_independientes_precio(df_test)

In [25]:
# ind_test[['escomercial',
#        'promedio_metros_tipo_propiedad','promedio_metros_cub_tipo_propiedad', 'tipo_propiedad_compartida',
#        'prop_frecuente', 'zona','top_provincia', 'es_ciudad_centrica',
#        'promedio_metros_totales_provincia',
#        'promedio_metros_cubiertos_provincia', 'porcentaje_metros',
#        'diferencia_metros', 'intervalo_metros_totales',
#        'intervalo_metros_cubiertos', 'anio', 'mes', 'dia', 'trimestre',
#        'escualas_centros_cercanos', 'delincuencia', 'turismo', 'es_antigua', 'cantidad_inquilinos']].head()

In [26]:
# # Verifico que no haya nans en los features
# ind_test.isnull().any()

In [27]:
# print(df_train.shape)
# print(ind.shape)
# print(df_test.shape)
# print(ind_test.shape)

In [28]:
# dep = features_dependientes_precio(df_test, df_train)

In [29]:
# dep = features_dependientes_precio(df_train, df_train)

In [30]:
# dep[['precio','promedio_precio_ciudad',
#        'varianza_precio_ciudad', 'promedio_id_zona', 'varianza_id_zona',
#        'promedio_precio_tipo_propiedad', 'promedio_por_mes',
#        'varianza_por_mes', 'promedio_precio_habitaciones',
#        'promedio_precio_habitaciones_banos_garages',
#        'promedio_precio_banos_garages', 'promedio_precio_booleanos', 'puntaje']].head()

In [31]:
# dep_test =  features_dependientes_precio(df_test, df_train)

In [32]:
# dep_test[['promedio_precio_ciudad',
#        'varianza_precio_ciudad', 'promedio_id_zona', 'varianza_id_zona',
#        'promedio_precio_tipo_propiedad', 'promedio_por_mes',
#        'varianza_por_mes', 'promedio_precio_habitaciones',
#        'promedio_precio_habitaciones_banos_garages',
#        'promedio_precio_banos_garages', 'promedio_precio_booleanos', 'puntaje']].head()

In [33]:
# print(df_train.shape)
# print(dep.shape)
# print(df_test.shape)
# print(dep_test.shape)

In [34]:
# df_train = pd.read_csv('data/train.csv')
# df_test = pd.read_csv('data/test.csv')

# df_train_f = features_dependientes_precio(df_train, df_train)
# df_test_f = features_dependientes_precio(df_test, df_train)

In [35]:
# df_train_f.head(10)

In [36]:
# df_test_f['promedio_precio_tipo_propiedad'].head(10).value_counts()

In [37]:
def tipo_de_propiedad_one_hot_encoding(df):
    '''Devuelve una copia del dataframe con nuevas columnas, 
    las nuevas columnas provienen de hacer OneHotEncondign respecto
    de la columna tipo de propiedad. Se retorna el dataframe recibido,
    con las nuevas columnas, y en segundo lugar, se retorna un array 
    de las nuevas columnas'''
    # Fusion
    df['tipodepropiedad'] = df['tipodepropiedad'].fillna('Otros')
    df[df['tipodepropiedad'] == 'Garage'] = 'Otros'
    df[df['tipodepropiedad'] == 'Hospedaje'] = 'Otros'
    df[df['tipodepropiedad'] == 'Lote'] = 'Otros'
    df[df['tipodepropiedad'] == 'Huerta'] = 'Otros'
    df[df['tipodepropiedad'] == 'Terreno industrial'] = 'Otros'
    df[df['tipodepropiedad'] == 'Nave industrial'] = 'Otros'
    df[df['tipodepropiedad'] == 'Departamento Compartido'] = 'Otros'
    df[df['tipodepropiedad'] == 'Local en centro comercial'] = 'Otros'
    df[df['tipodepropiedad'] == 'Rancho'] = 'Otros'
    # Espacios por guion bajo
    df[df['tipodepropiedad'] == 'Casa en condominio'] = 'Casa_en_condominio'
    df[df['tipodepropiedad'] == 'Local Comercial'] = 'Local_Comercial'
    df[df['tipodepropiedad'] == 'Oficina comercial'] = 'Oficina_comercial'
    df[df['tipodepropiedad'] == 'Bodega comercial'] = 'Bodega_comercial'
    df[df['tipodepropiedad'] == 'Terreno comercial'] = 'Terreno_comercial'
    df[df['tipodepropiedad'] == 'Casa uso de suelo'] = 'Casa_uso_de_suelo'
    df[df['tipodepropiedad'] == 'Quinta Vacacional'] = 'Quinta_Vacacional'
    df[df['tipodepropiedad'] == 'Inmuebles productivos urbanos'] = 'Inmuebles_productivos_urbanos'
    
    posibles_categorias = ['Otros', 'Casa_en_condominio', 'Local_Comercial', 'Oficina_comercial', 
                          'Bodega_comercial', 'Terreno_comercial', 'Casa_uso_de_suelo', 'Quinta_Vacacional',
                           'Inmuebles_productivos_urbanos']
        
    encoded_tipo_propiedad = pd.get_dummies(df[['id', 'tipodepropiedad']], columns=['tipodepropiedad'])
    
    new_columns = []
    for column in encoded_tipo_propiedad.columns:
        if "tipodepropiedad_" in column:
            new_columns.append(column)
    
    for new_colum in posibles_categorias:
        encoded_tipo_propiedad = encoded_tipo_propiedad[encoded_tipo_propiedad['id'] != new_colum]
    
    return pd.merge(df, encoded_tipo_propiedad, on='id'), new_columns
#     return encoded_tipo_propiedad, new_columns


In [38]:
# df_train = pd.read_csv('./data/train.csv')

In [ ]:
# df, arr = tipo_de_propiedad_one_hot_encoding(df_train)
# display(df)
# display(df.columns)
# df['titulo'].value_counts()

In [10]:
def columna_a_ohe(df, col, N=0, df_aux=None, devolver_cols=False):
    '''Dado un dataframe df, aplica One Hot Encoding a la columna del mismo.
       Para todas las categorias con menos de N elementos, las agrupa bajo 
    una nueva categoria llamdo 'otro'.
       Si df_aux no es nulo, se llenaran tambien columnas de las categorias que no
    se encuentren en el dataframe original, asi ambos tienen las mismas dimensiones.
       Si devolver_cols es True, ademas de devolver el dataframe, devolvera las
    nuevas columnas creadas por OHE.'''
    if df_aux is None:
        df_aux = pd.DataFrame(columns=df.columns)

    df_aux = df_aux.copy()
    df = df.copy()
    
    df_full = pd.concat([df[[col]], df_aux[[col]]])
    df_full.loc[df_full.groupby(col)[col].transform('size') < N, col] = 'otro'
    
    df_full_ohe = pd.get_dummies(df_full, columns=[col])
    # Eliminar caracteres utf-8
    df_full_ohe.columns = df_full_ohe.columns.str.encode('ascii', 'ignore').str.decode('ascii')
    df_ohe = df_full_ohe.iloc[0:len(df.values)]
    
    df = pd.concat([df, df_ohe], axis=1, join='inner')
    
    if devolver_cols:
        return df, list(df_full_ohe.columns)
    return df

In [5]:
# import pandas as pd
# df_train = pd.read_csv('./data/train.csv')
# df_test = pd.read_csv('./data/test.csv')

In [11]:
# df_train_ohe, new_cols_train = columna_a_ohe(df_train, 'tipodepropiedad', 100, df_test, True)

In [ ]:
# df_test_ohe, new_cols = columna_a_ohe(df_test, 'tipodepropiedad', 2000, df_train, True)

In [12]:
# df_train_ohe[['tipodepropiedad'] + new_cols_train]

,tipodepropiedad,tipodepropiedad_Apartamento,tipodepropiedad_Bodega comercial,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Departamento Compartido,tipodepropiedad_Duplex,tipodepropiedad_Edificio,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Local Comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Oficina comercial,tipodepropiedad_Otros,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Villa,tipodepropiedad_otro
0,Apartamento,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Casa en condominio,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Casa,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Casa,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Apartamento,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,Casa,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
239996,Casa,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
239997,Apartamento,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
239998,Casa,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
